In [1]:
import pygame
import time

def main():
    pygame.init()
    pygame.joystick.init()

    joystick_count = pygame.joystick.get_count()
    if joystick_count == 0:
        print("没有检测到手柄，确保已连接")
        return

    # 初始化所有找到的手柄
    joysticks = []
    for i in range(joystick_count):
        joy = pygame.joystick.Joystick(i)
        joy.init()
        joysticks.append(joy)
        print(f"检测到手柄 {i}:")
        print("  名字:", joy.get_name())
        print("  GUID:", joy.get_guid()) # GUID 是更唯一的标识符
        print("  轴 (axes):", joy.get_numaxes())
        print("  按钮 (buttons):", joy.get_numbuttons())
        print("  帽子 (hats):", joy.get_numhats())
        
    print("\n--- 开始监听事件 ---")

    # 设置一个小窗口（对于事件处理是必要的）
    screen = pygame.display.set_mode((200, 100))
    pygame.display.set_caption("Joystick Test")

    try:
        while True:
            # 1. 处理事件队列
            for event in pygame.event.get():
                # print("Got event:", event) # debug 输出所有事件
                if event.type == pygame.QUIT:
                    print("退出事件")
                    return
                
                # --- 手柄事件处理 ---
                # event.instance_id 会告诉你这是哪个手柄实例触发的事件
                
                if event.type == pygame.JOYAXISMOTION:
                    # 添加一个阈值，忽略摇杆的轻微抖动（死区）
                    if abs(event.value) > 0.1:
                        print(f"手柄 {event.instance_id} | 轴 {event.axis} 移动, 值 = {event.value:.3f}")
                elif event.type == pygame.JOYBUTTONDOWN:
                    print(f"手柄 {event.instance_id} | 按钮 {event.button} 按下")
                elif event.type == pygame.JOYBUTTONUP:
                    print(f"手柄 {event.instance_id} | 按钮 {event.button} 松开")
                elif event.type == pygame.JOYHATMOTION:
                    print(f"手柄 {event.instance_id} | 帽子 {event.hat} 移动, 值 = {event.value}")

            time.sleep(0.02) # 稍微减少延迟以便更快响应
    except KeyboardInterrupt:
        print("用户中断退出")
    finally:
        pygame.quit()

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
检测到手柄 0:
  名字: Xbox Series X Controller
  GUID: 0300509d5e040000120b000009056800
  轴 (axes): 6
  按钮 (buttons): 15
  帽子 (hats): 0

--- 开始监听事件 ---
退出事件


In [2]:
import pygame
import time

# --- 新增：设置常量 ---
# 屏幕尺寸
SCREEN_WIDTH = 500
SCREEN_HEIGHT = 300
# 颜色
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

def main():
    pygame.init()
    pygame.joystick.init()
    
    # --- 新增：初始化字体和时钟 ---
    font = pygame.font.Font(None, 74) # 用于显示页码的字体
    clock = pygame.time.Clock()      # 用于控制刷新率

    joystick_count = pygame.joystick.get_count()
    if joystick_count == 0:
        print("没有检测到手柄，确保已连接")
        return

    # 初始化所有找到的手柄
    joysticks = []
    for i in range(joystick_count):
        print(i)
        joy = pygame.joystick.Joystick(i)
        joy.init()
        joysticks.append(joy)
        print(f"检测到手柄 {i}: {joy.get_name()}")
        
    print("\n--- 开始监听事件 ---")
    print("使用方向键(D-pad)、肩键(L1/R1)或左摇杆左右拨动来翻页。")


    # 设置窗口
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption("手柄翻页测试")

    # --- 1. 创建“页面”状态 ---
    current_page = 1
    total_pages = 100
    
    # --- 新增：用于处理摇杆的变量 ---
    # 这个布尔值确保你拨动一次摇杆只翻一页，而不是连续翻页
    axis_moved = False 

    running = True
    try:
        while running:
            # --- 2. 映射输入到动作 (事件处理) ---
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False # 优雅地退出循环
                
                # --- A. 使用方向键 (Hat) 翻页 ---
                if event.type == pygame.JOYHATMOTION:
                    # event.value 是一个元组 (x, y)
                    # (1, 0) 是右, (-1, 0) 是左
                    hat_x, hat_y = event.value
                    if hat_x == 1:
                        current_page += 1
                        print("方向键: 下一页")
                    elif hat_x == -1:
                        current_page -= 1
                        print("方向键: 上一页")
                
                # --- B. 使用按钮 (Buttons) 翻页 ---
                # 通常，按钮4是L1/LB，按钮5是R1/RB。你的手柄可能不同。
                if event.type == pygame.JOYBUTTONDOWN:
                    if event.button == 5: # R1/RB
                        current_page += 1
                        print("按钮: 下一页")
                    elif event.button == 4: # L1/LB
                        current_page -= 1
                        print("按钮: 上一页")
                        
                # --- C. 使用摇杆 (Axis) 翻页 ---
                if event.type == pygame.JOYAXISMOTION:
                    # 通常，轴0是左摇杆的左右方向
                    if event.axis == 0: 
                        # 向右拨动 (值接近 1.0)
                        if event.value > 0.8 and not axis_moved:
                            current_page += 1
                            print("摇杆: 下一页")
                            axis_moved = True # 标记为已移动，防止连续触发
                        # 向左拨动 (值接近 -1.0)
                        elif event.value < -0.8 and not axis_moved:
                            current_page -= 1
                            print("摇杆: 上一页")
                            axis_moved = True # 标记为已移动
                        # 摇杆回中
                        elif abs(event.value) < 0.2:
                            axis_moved = False # 回到中间时重置标志

            # --- 核心逻辑：确保页码不会超出范围 ---
            if current_page > total_pages:
                current_page = total_pages
            if current_page < 1:
                current_page = 1
            
            # --- 3. 显示当前状态 (绘制屏幕) ---
            # 用黑色填充背景
            screen.fill(BLACK)
            
            # 准备要显示的文本
            text_to_show = f"Page {current_page} / {total_pages}"
            text_surface = font.render(text_to_show, True, WHITE)
            
            # 将文本居中显示
            text_rect = text_surface.get_rect(center=(SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2))
            
            # 在屏幕上绘制文本
            screen.blit(text_surface, text_rect)
            
            # 更新整个屏幕
            pygame.display.flip()
            
            # 控制游戏循环的频率，比如每秒60帧
            clock.tick(60)

    except KeyboardInterrupt:
        print("用户中断退出")
    finally:
        pygame.quit()

if __name__ == "__main__":
    main()

0
检测到手柄 0: Xbox Series X Controller
1
检测到手柄 1: Controller

--- 开始监听事件 ---
使用方向键(D-pad)、肩键(L1/R1)或左摇杆左右拨动来翻页。
按钮: 下一页
按钮: 下一页
按钮: 上一页
按钮: 下一页


In [1]:
import pygame
import time
import pyautogui # 1. 导入新库

# --- 安全提示 ---
# pyautogui 可以控制你的电脑，如果脚本失控会很危险。
# 如果程序出现问题，快速将鼠标移动到屏幕的左上角 (0, 0) 可以触发 pyautogui 的 Failsafe 机制，强制停止程序。
pyautogui.FAILSAFE = True

def main():
    pygame.init()
    pygame.joystick.init()

    joystick_count = pygame.joystick.get_count()
    if joystick_count == 0:
        print("没有检测到手柄，确保已连接")
        return

    # 初始化手柄
    joy = pygame.joystick.Joystick(0)
    joy.init()
    print(f"已连接手柄: {joy.get_name()}")
    print("\n--- 开始监听，现在你可以切换到其他应用了 ---")
    print("使用方向键、肩键或左摇杆来模拟键盘的 '左/右箭头' 或 'PageUp/PageDown'")
    print("!!! 安全提示: 将鼠标移动到屏幕左上角可紧急停止程序 !!!")

    # 2. 窗口现在只是一个后台监听器，可以做得很小
    screen = pygame.display.set_mode((400, 150))
    pygame.display.set_caption("Joystick Global Controller")
    font = pygame.font.Font(None, 36)
    clock = pygame.time.Clock()

    axis_moved = False
    running = True
    try:
        while running:
            # 事件处理
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

                # A. 使用方向键 (Hat) 翻页 -> 模拟 PageUp/PageDown
                if event.type == pygame.JOYHATMOTION:
                    hat_x, hat_y = event.value
                    if hat_x == 1:
                        print("方向键: 模拟 'Page Down'")
                        pyautogui.press('pagedown') # 3. 模拟按下 PageDown 键
                    elif hat_x == -1:
                        print("方向键: 模拟 'Page Up'")
                        pyautogui.press('pageup')   # 模拟按下 PageUp 键
                
                # B. 使用按钮 (Buttons) 翻页 -> 模拟 左/右 箭头
                if event.type == pygame.JOYBUTTONDOWN:
                    if event.button == 5: # R1/RB
                        print("按钮: 模拟 '右箭头'")
                        pyautogui.press('right') # 模拟按下右箭头
                    elif event.button == 4: # L1/LB
                        print("按钮: 模拟 '左箭头'")
                        pyautogui.press('left')  # 模拟按下左箭头
                        
                # C. 使用摇杆 (Axis) 翻页 -> 模拟 左/右 箭头
                if event.type == pygame.JOYAXISMOTION:
                    if event.axis == 0: # 左摇杆左右
                        if event.value > 0.8 and not axis_moved:
                            print("摇杆: 模拟 '右箭头'")
                            pyautogui.press('right') # 模拟按下右箭头
                            axis_moved = True
                        elif event.value < -0.8 and not axis_moved:
                            print("摇杆: 模拟 '左箭头'")
                            pyautogui.press('left')  # 模拟按下左箭头
                            axis_moved = True
                        elif abs(event.value) < 0.2:
                            axis_moved = False

            # 4. 在窗口上显示状态信息，而不是页码
            screen.fill((0, 0, 0))
            text_surface = font.render("手柄监听中...", True, (255, 255, 255))
            text_rect = text_surface.get_rect(center=(200, 75))
            screen.blit(text_surface, text_rect)
            pygame.display.flip()
            
            clock.tick(60)

    except pyautogui.FailSafeException:
        print("安全机制触发，程序已停止。")
    except KeyboardInterrupt:
        print("用户中断退出")
    finally:
        pygame.quit()

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
已连接手柄: Xbox Series X Controller

--- 开始监听，现在你可以切换到其他应用了 ---
使用方向键、肩键或左摇杆来模拟键盘的 '左/右箭头' 或 'PageUp/PageDown'
!!! 安全提示: 将鼠标移动到屏幕左上角可紧急停止程序 !!!


In [3]:
import pygame
import time
import pyautogui

# --- 安全提示 ---
# pyautogui 可以控制你的电脑，如果脚本失控会很危险。
# 如果程序出现问题，快速将鼠标移动到屏幕的左上角 (0, 0) 可以触发 pyautogui 的 Failsafe 机制，强制停止程序。
pyautogui.FAILSAFE = True

# --- 配置项 ---
AXIS_THRESHOLD = 0.8  # 摇杆触发的灵敏度阈值
AXIS_COOLDOWN = 0.2   # 摇杆连续触发的冷却时间（秒），值越小翻页越快

def main():
    pygame.init()
    pygame.joystick.init()

    joystick_count = pygame.joystick.get_count()
    if joystick_count == 0:
        print("没有检测到手柄，确保已连接")
        return

    # 初始化手柄
    joy = pygame.joystick.Joystick(0)
    joy.init()
    print(f"已连接手柄: {joy.get_name()}")
    print("\n--- 开始监听，现在你可以切换到其他应用了 ---")
    print("使用方向键、肩键或左摇杆来模拟键盘的 '左/右箭头' 或 'PageUp/PageDown'")
    print(f"!!! 安全提示: 将鼠标移动到屏幕左上角({(0,0)})可紧急停止程序 !!!")

    # 窗口现在只是一个后台监听器，可以做得很小
    screen = pygame.display.set_mode((450, 150))
    pygame.display.set_caption("Joystick Global Controller (后台监听中)")
    font = pygame.font.Font(None, 30) # 使用系统默认字体
    clock = pygame.time.Clock()

    # 用于处理摇杆连续触发的变量
    last_axis_press_time = 0 
    
    running = True
    try:
        while running:
            # --- 事件处理 ---
            # 必须调用 pygame.event.get() 来让 pygame 内部处理事件，否则窗口会无响应
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

                # A. 使用方向键 (Hat) 翻页 -> 模拟 PageUp/PageDown
                if event.type == pygame.JOYHATMOTION:
                    hat_x, hat_y = event.value
                    if hat_x == 1:
                        print("方向键: 模拟 'Page Down'")
                        pyautogui.press('pagedown')
                    elif hat_x == -1:
                        print("方向键: 模拟 'Page Up'")
                        pyautogui.press('pageup')
                
                # B. 使用按钮 (Buttons) 翻页 -> 模拟 左/右 箭头
                if event.type == pygame.JOYBUTTONDOWN:
                    if event.button == 5: # R1/RB
                        print("按钮: 模拟 '右箭头'")
                        pyautogui.press('right')
                    elif event.button == 4: # L1/LB
                        print("按钮: 模拟 '左箭头'")
                        pyautogui.press('left')
                        
            # --- 轮询处理摇杆（为了实现按住连续触发） ---
            # 从事件循环中移出，改为每帧检查状态
            current_time = time.time()
            if current_time - last_axis_press_time > AXIS_COOLDOWN:
                # 获取左摇杆X轴的值
                axis_x = joy.get_axis(0) 
                
                if axis_x > AXIS_THRESHOLD:
                    print("摇杆: 模拟 '右箭头'")
                    pyautogui.press('right')
                    last_axis_press_time = current_time # 重置计时器
                elif axis_x < -AXIS_THRESHOLD:
                    print("摇杆: 模拟 '左箭头'")
                    pyautogui.press('left')
                    last_axis_press_time = current_time # 重置计时器

            # --- 窗口显示 ---
            screen.fill((20, 20, 40)) # 深蓝色背景
            text_lines = [
                "手柄监听中...",
                "请点击其他窗口以控制它",
                "移动鼠标到左上角可停止"
            ]
            for i, line in enumerate(text_lines):
                 text_surface = font.render(line, True, (200, 255, 200))
                 text_rect = text_surface.get_rect(center=(screen.get_width() / 2, 40 + i * 40))
                 screen.blit(text_surface, text_rect)
            
            pygame.display.flip()
            
            clock.tick(60)

    except pyautogui.FailSafeException:
        print("安全机制触发，程序已停止。")
    except KeyboardInterrupt:
        print("用户中断退出 (Ctrl+C)")
    finally:
        pygame.quit()
        print("程序已退出。")

if __name__ == "__main__":
    main()

已连接手柄: Xbox Series X Controller

--- 开始监听，现在你可以切换到其他应用了 ---
使用方向键、肩键或左摇杆来模拟键盘的 '左/右箭头' 或 'PageUp/PageDown'
!!! 安全提示: 将鼠标移动到屏幕左上角((0, 0))可紧急停止程序 !!!
用户中断退出 (Ctrl+C)
程序已退出。
